In [35]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
import pandas as pd
#sys.path.append("/home/CVShare/Jeni/hts/machine_learning/sampling_with_data_cleaning")
import sampling_with_data_cleaning as sdc
from utils import *
import sampling_with_data_cleaning as sdc

### FNN on normal features

In [36]:
findex = 2
x,y = get_features(findex,train=True,cleaned=True)
print y.shape
print sum(y)
y = y.astype(int)
print sum(y)
x,y = shuffle(x,y)

(7881,)
129.0
129


In [37]:
no_ex,ip_dim = x.shape
print no_ex,ip_dim

7881 153


In [38]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,500)
        self.l2 = nn.Linear(500,500)
        self.l3 = nn.Linear(500,500)
        self.l4 = nn.Linear(500,100)
        self.l5 = nn.Linear(100,30)
        self.l6 = nn.Linear(30,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        #x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = F.leaky_relu(self.l5(x))
        x = (self.l6(x))
        
        return x

In [44]:
max_percent = 0.5
min_fp = no_ex*0.2
#for i in range(1,6):
if(1==1):
    val_iter = 0
    
   
    weights_array = [1]
    #weights_array = np.linspace(10,40,10)
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2000):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(x,y,batch_size=200,indices=ind)
            
            model_op = mymlp(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if(ep%30==29):
                print(loss.data[0])

0.105780228972
0.0783683657646
0.0376045443118
0.106086723506
0.0839707776904
0.064403116703
0.0986544042826
0.0642578452826
0.0667999535799
0.0855871140957
0.0905609354377
0.097773462534
0.0913430973887
0.125619396567
0.0441837161779
0.0226480867714
0.0401112698019
0.0317909680307
0.0739508420229
0.100209616125
0.0699261948466
0.0512844100595
0.054569683969
0.0657065957785
0.0816217958927
0.135687708855
0.0573780499399
0.0346733294427
0.0295233707875
0.0528658516705
0.019427401945
0.0396709479392
0.116745777428
0.0424507521093
0.0680215060711
0.0678362473845
0.0326717533171
0.0387614704669
0.0570288151503
0.0174219794571
0.0392023511231
0.0385765470564
0.0350663363934
0.00908144563437
0.0224525686353
0.0470472685993
0.0181341338903
0.0381902940571
0.0338332019746
0.0417667850852
0.0169199630618
0.0440876297653
0.02527474612
0.0354266315699
0.0194860361516
0.0346940197051
0.0400318577886
0.0924149602652
0.0124143026769
0.00676574697718
0.0435386672616
0.00329604139552
0.0131255909801
0

In [45]:
if(1==1):
    if(1==1):
        ## After training check on ORINGAL data
        print('Training over')
        xch = Variable(torch.cuda.FloatTensor(x))
        train_op = mymlp(xch)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
      
        cf = metrics.confusion_matrix(y,pred_labels).ravel()
        [tn,fp,fn,tp] = cf
        print('tn, fp, fn, tp: ',cf)
        percent = float(float(tp)/float(tp+fn))

Training over
('tn, fp, fn, tp: ', array([7746,    6,   39,   90]))


In [46]:
model_path = os.getcwd() + '/boost_clean_a' + str(findex)
torch.save(mymlp.state_dict(),model_path)

In [47]:
## Just checking on test data

In [48]:
xtest,ytest = get_features(findex,train=False)
ytest = ytest.astype(int)
xtest = Variable(torch.cuda.FloatTensor(xtest))
testmodel = c_mlp().cuda()
testmodel.load_state_dict(torch.load(model_path))
test_op = testmodel(xtest).cpu().data.numpy()
test_op = np.argmax(test_op,axis=1)
cf = metrics.confusion_matrix(ytest,test_op).ravel()
[tn,fp,fn,tp] = cf
print('tn, fp, fn, tp: ',cf)
percent = float(float(tp)/float(tp+fn))

('tn, fp, fn, tp: ', array([1983,    8,    5,    0]))


### Find out +ve samples on which this fails

In [49]:
#y_hack = np.zeros(y.shape)
#y_hack[y==1] = 5
y = y.reshape(y.shape[0],)
next_t = y-pred_labels  #next_ind will be 1 for which pred_labels have predicted inactive
next_ind = np.argwhere(next_t == 1)
print next_ind.shape
xtmp = np.delete(x,next_ind,axis=0)
ytmp = np.delete(y,next_ind,axis=0)
print xtmp.shape, ytmp.shape

(39, 1)
(7842, 153) (7842,)


In [50]:
xnew,ynew = sdc.clean_data(xtmp,ytmp,k=8)

here
Adding 97 new samples
removing 26 samples
[2349 1990 6047 3704 1507 6230 2916 3372  553 5209 6291 7775 3161 3649 6520
 2141 6718 7602 7178 2462 1923 6734 6099 1704 1889 5051]


In [51]:
ynew = ynew.astype(int)

In [52]:
max_percent = 0.5
no_ex = xnew.shape[0]
min_fp = no_ex*0.2
#for i in range(1,6):
if(1==1):
    val_iter = 0
    
   
    weights_array = [1]
    #weights_array = np.linspace(10,40,10)
    for i,w in enumerate(weights_array): 
        mymlp_b = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp_b.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2500):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(xnew,ynew,batch_size=150,indices=ind)
            
            model_op = mymlp_b(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if(ep%30==29):
                print(loss.data[0])

0.100020326674
0.147966355085
0.108428575099
0.114323116839
0.0606371499598
0.0358465053141
0.10158097744
0.12946844101
0.117929942906
0.181353494525
0.0705535411835
0.136251226068
0.11305937171
0.0509777069092
0.038243688643
0.0643781498075
0.0547373145819
0.0607002191246
0.0843674987555
0.0612891949713
0.0811889842153
0.0480520166457
0.0728653892875
0.0858905315399
0.0600149780512
0.0659052506089
0.02327625826
0.0676613748074
0.0795780420303
0.0885193869472
0.0381510481238
0.045357093215
0.0873620435596
0.0330195501447
0.099658280611
0.0423472225666
0.0139985848218
0.0215290896595
0.0222295317799
0.022947544232
0.0215113647282
0.0154647873715
0.0248546954244
0.0136203793809
0.0321335569024
0.00886239111423
0.0184576343745
0.00960175786167
0.0327467806637
0.0141112627462
0.0423163212836
0.0060911718756
0.0174268316478
0.00693074567243
0.00227925693616
0.0192413870245
0.0121884588152
0.0122456792742
0.0265737902373
0.0195202063769
0.0210574511439
0.00402791565284
0.00865935720503
0.007

In [53]:
if(1==1):
    if(1==1):
        ## After training check on ORINGAL data
        print('Training over')
        xch = Variable(torch.cuda.FloatTensor(xnew))
        train_op = mymlp_b(xch)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
      
        cf = metrics.confusion_matrix(ynew,pred_labels).ravel()
        [tn,fp,fn,tp] = cf
        print('tn, fp, fn, tp: ',cf)
        percent = float(float(tp)/float(tp+fn))

Training over
('tn, fp, fn, tp: ', array([7724,    2,    8,  179]))


In [54]:
model_path = os.getcwd() + '/boost_clean_b' + str(findex)
torch.save(mymlp_b.state_dict(),model_path)

In [55]:
## Test after training on both
xtest,ytest = get_features(findex,train=False)
xtest = Variable(torch.cuda.FloatTensor(xtest))
opa = mymlp(xtest).data.cpu().numpy()
pred1 = np.argmax(opa,axis=1)
opb = mymlp_b(xtest).data.cpu().numpy()
pred2 = np.argmax(opb,axis=1)
#opc = mymlp_c(xtest).data.cpu().numpy()
#pred3 = np.argmax(opc,axis=1)

final_pred = pred1 + pred2# + pred3
final_pred[final_pred>0] = 1

cf = metrics.confusion_matrix(ytest,final_pred).ravel()
[tn,fp,fn,tp] = cf
print('tn, fp, fn, tp: ',cf)
percent = float(float(tp)/float(tp+fn))

('tn, fp, fn, tp: ', array([1983,    8,    5,    0]))


In [56]:
cf = metrics.confusion_matrix(ytest,pred2).ravel()
[tn,fp,fn,tp] = cf
print('tn, fp, fn, tp: ',cf)
percent = float(float(tp)/float(tp+fn))

('tn, fp, fn, tp: ', array([1986,    5,    5,    0]))


In [57]:
ynew = ynew.reshape(ynew.shape[0],)
next_t = ynew-pred_labels  #next_ind will be 1 for which pred_labels have predicted inactive
next_ind = np.argwhere(next_t == 1)
print next_ind.shape
xtmp = np.delete(xnew,next_ind,axis=0)
ytmp = np.delete(ynew,next_ind,axis=0)
print xtmp.shape, ytmp.shape

(8, 1)
(7905, 153) (7905,)


In [58]:
xnew2,ynew2 = sdc.clean_data(xtmp,ytmp,k=8)

here
Adding 30 new samples
removing 15 samples
[7453  526 5064 6541 2390 2943 7595 7688 3075  732 3489 3174 5834 3408 1513]


In [59]:
ynew2 = ynew2.astype(int)

In [63]:
max_percent = 0.5
no_ex = xnew2.shape[0]
min_fp = no_ex*0.2
#for i in range(1,6):
if(1==1):
    val_iter = 0
    
   
    weights_array = [1]
    #weights_array = np.linspace(10,40,10)
    for i,w in enumerate(weights_array): 
        mymlp_c = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp_c.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2000):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(xnew2,ynew2,batch_size=150,indices=ind)
            
            model_op = mymlp_c(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if(ep%30==29):
                print(loss.data[0])

0.161867260933
0.0972013548017
0.138442531228
0.0855605974793
0.0636093094945
0.052924849093
0.0971305221319
0.1013789922
0.107973277569
0.0533413179219
0.0641384497285
0.0764310657978
0.0609779804945
0.0526796393096
0.0997552722692
0.0389297902584
0.0503053888679
0.0494070313871
0.0726574733853
0.0770090892911
0.0285440068692
0.0563500486314
0.0589921623468
0.0229014977813
0.0512113943696
0.0121485311538
0.0497678443789
0.0151376705617
0.0577541589737
0.0596417933702
0.0179365240037
0.0266345832497
0.0446280017495
0.0293675065041
0.028711521998
0.0277307517827
0.0357581339777
0.0212229061872
0.0138394720852
0.0246779918671
0.0187380667776
0.0161620508879
0.013542030938
0.0134357418865
0.0218046661466
0.00901272334158
0.00233350438066
0.0106224957854
0.00855824071914
0.00510103069246
0.016798870638
0.0132543118671
0.00385350384749
0.00756954448298
0.0121798114851
0.00736415060237
0.012183717452
0.0188790280372
0.00490781804547
0.0107836714014
0.00454638944939
0.00762637844309
0.0042678

In [64]:
model_path = os.getcwd() + '/boost_clean_c' + str(findex)
torch.save(mymlp_c.state_dict(),model_path)

In [65]:
## Test after training on both
xtest,ytest = get_features(findex,train=False)
xtest = Variable(torch.cuda.FloatTensor(xtest))
opa = mymlp(xtest).data.cpu().numpy()
pred1 = np.argmax(opa,axis=1)
opb = mymlp_b(xtest).data.cpu().numpy()
pred2 = np.argmax(opb,axis=1)
opc = mymlp_c(xtest).data.cpu().numpy()
pred3 = np.argmax(opc,axis=1)

final_pred = pred1 + pred2 + pred3
final_pred[final_pred>0] = 1

cf = metrics.confusion_matrix(ytest,final_pred).ravel()
[tn,fp,fn,tp] = cf
print('tn, fp, fn, tp: ',cf)
percent = float(float(tp)/float(tp+fn))

('tn, fp, fn, tp: ', array([1981,   10,    5,    0]))
